In [11]:
import gensim
import os
import numpy as np
import pandas as pd

In [9]:
!pip install kaggle

In [10]:
!kaggle datasets download khulasasndh/game-of-thrones-books

Dataset URL: https://www.kaggle.com/datasets/khulasasndh/game-of-thrones-books
License(s): unknown
  0% 0.00/3.71M [00:00<?, ?B/s]
100% 3.71M/3.71M [00:00<00:00, 191MB/s]


In [14]:
!unzip game-of-thrones-books.zip

Archive:  game-of-thrones-books.zip
  inflating: 001ssb.txt              
  inflating: 002ssb.txt              
  inflating: 003ssb.txt              
  inflating: 004ssb.txt              
  inflating: 005ssb.txt              


In [16]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [17]:
from nltk.tokenize import sent_tokenize
from gensim.utils import simple_preprocess

In [19]:
story = []

for filename in os.listdir('data'):
    with open(os.path.join('data', filename), 'r', encoding='latin-1') as f:
        corpus = f.read()
    raw_sent = sent_tokenize(corpus)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))

In [23]:
len(story)

145020

In [20]:
model = gensim.models.Word2Vec(
    window = 10,
    min_count = 2,
    workers = 4
)

In [22]:
model.build_vocab(story)

In [24]:
model.train(story, total_examples=model.corpus_count, epochs=model.epochs)

(6570553, 8628190)

In [25]:
model.wv.most_similar('daenerys')

[('stormborn', 0.7510025501251221),
 ('targaryen', 0.7169267535209656),
 ('myrcella', 0.697087824344635),
 ('queen', 0.6896904110908508),
 ('princess', 0.6793946623802185),
 ('viserys', 0.6763966083526611),
 ('unburnt', 0.6600896120071411),
 ('margaery', 0.6600666642189026),
 ('dorne', 0.6532927751541138),
 ('rhaegar', 0.6420192122459412)]

In [26]:
model.wv.most_similar('jon')

[('theon', 0.741598904132843),
 ('sam', 0.7180694937705994),
 ('bran', 0.6600196957588196),
 ('davos', 0.6347207427024841),
 ('ygritte', 0.6303863525390625),
 ('grenn', 0.6291360855102539),
 ('tormund', 0.6183556914329529),
 ('asha', 0.6121481657028198),
 ('pyp', 0.6066064238548279),
 ('qhorin', 0.6030473113059998)]

In [28]:
model.wv.doesnt_match(['jon','robb','arya','sansa','bran'])

'jon'

In [29]:
model.wv.doesnt_match(['cersei', 'jaime', 'bronn', 'tyrion'])

'bronn'

In [30]:
model.wv.get_normed_vectors()

array([[-0.0508897 , -0.03025898,  0.13969569, ...,  0.04861248,
        -0.14164265,  0.03579903],
       [-0.04876833, -0.01257843, -0.04476786, ..., -0.08079939,
        -0.14620098,  0.13228615],
       [ 0.11279394, -0.05987245, -0.12484107, ..., -0.03565897,
         0.08908126, -0.18783005],
       ...,
       [ 0.21261695,  0.02918598, -0.02021493, ..., -0.05097864,
         0.13852412, -0.01579188],
       [ 0.07110929,  0.0491496 ,  0.04301296, ...,  0.08812058,
         0.00195896,  0.08323693],
       [-0.18642217,  0.09974889,  0.02291651, ..., -0.1082442 ,
        -0.02704804,  0.07405423]], dtype=float32)

In [31]:
y = model.wv.index_to_key

In [32]:
len(y)

17453

In [33]:
from sklearn.decomposition import PCA

In [34]:
pca = PCA(n_components = 3)

In [35]:
X = pca.fit_transform(model.wv.get_normed_vectors())

In [36]:
X[:5]

array([[ 0.14500369, -0.59656006,  0.02627565],
       [ 0.16197775, -0.31917924, -0.02706589],
       [-0.3010134 , -0.5820193 , -0.22424458],
       [ 0.01468322, -0.374925  ,  0.14037158],
       [-0.11624364, -0.5469026 , -0.28684166]], dtype=float32)

In [37]:
X.shape

(17453, 3)

In [38]:
import plotly.express as px

In [40]:
fig = px.scatter_3d(X[:100], x = 0, y = 1, z = 2, color = y[:100])
fig.show()